# 3. Starting and monitoring Dask

A very nice feature of Dask, is that it allows to use the almost exact same code to perform distributed computing on a laptop, in the cloud or on a cluster. This is made possible through the ```dask.distributed``` module, which takes care of setting up two important components: the cluster and the client. The cluster represents the computing resources reserved (cores on a laptop, jobs on a cluster etc.) while the client is the interface that allows us to access to that cluster.

## Cluster creation
First we create a cluster, here a local cluster on this machine, but it could also be a cluster e.g. on an HPC facility. We can specify many options here, in particular the number of workers (separate Python processes), the threads per worker etc. There are many options to configure computing resources that go beyond this introduction (see e.g. [here](https://distributed.dask.org/en/latest/worker.html) for more details).

In [ ]:
from dask.distributed import LocalCluster

In [ ]:
cluster = LocalCluster(n_workers=1, threads_per_worker=2)

In [ ]:
cluster

## Connecting to cluster via client
Now we can connect a client to our cluster to be able to acutally use it and submit computations. This is so to say our interface to the cluster:

In [ ]:
from dask.distributed import Client

In [ ]:
client = Client(cluster)

Here is our client:

In [ ]:
client

We see on the right a summary of the status of our cluster and on the left, two important addresses:

- The dashboard address (of the type http://127.0.0.1:8787/status) leads us to a dashboard where we can monitor the activity of the cluster. The Dask extension for Jupyterlab allows us to have acces to the monitoring dashborad directly within Jupyterlab. We can just choose the Dask icon on the left and use the Dashboard address to access all panels. **Note that when using a Jupyterhub you have to change the address to https://address-of-hub.ch/user/your-user-name/proxy/8787**
 
- The scheduler address allows us to create additional workers, e.g. directly in the terminal, by pointing them to the correct scheduler and typing:

    ```
    dask-worker "tcp://127.0.0.1:55323"
    ```

On top of many methods and attributes, the ```client``` also offers an interactive interface to the cluster:

In [ ]:
client.cluster

Here we can interactively select the number of workers and say if we want it to be automatically adjusted.

## Using Dask extension

An alternative to create a cluster, is to use the Dask extension. If you click on "+NEW", a cluster is automatically created. To use it in your notebook simple click on the "<>" button which will add the client to your current notebook.

In [ ]:
from IPython.display import HTML

HTML("""
<video width="800" controls>
  <source src="images/cluster_extension.mp4" type="video/mp4">
</video>
""")

## Example
We are going to see all these pieces in action now using a simple example handling numpy arrays.

In [ ]:
from dask import delayed
import numpy as np

In [ ]:
all_vals = []
for i in range(10):
    ar = delayed(np.random.randint)(1,100,(1000,1000,10))
    ar = delayed(np.log)(ar)#[::2,::2,0]
    mean_val = delayed(np.mean)(ar)
    all_vals.append(mean_val)
maxval = delayed(np.max)(all_vals)

In [ ]:
maxval.visualize()

In [ ]:
maxval.compute()

Now we do the same operation for a lot more iterations:

In [ ]:
all_vals = []
for i in range(1000):
    ar = delayed(np.random.randint)(1,100,(1000,1000,10))
    ar = delayed(np.log)(ar)#[::2,::2,0]
    mean_val = delayed(np.mean)(ar)
    all_vals.append(mean_val)
maxval = delayed(np.max)(all_vals)

In [ ]:
cluster

In [ ]:
maxval.compute()

Since this is very slow, we can now stop the operation, add a worker and restart.

Alternatively, as described above, to avoid stopping the calculation, we can dymamically assing new workers to the scheduler:

In [ ]:
client

## Asynchronous calculation

We have just seen how to add new workers to a running computation. We had to do that through the command line, because the computation is blocking the execution of other tasks until completion. We can go around this by executing the tasks asynchronously using the ```client.compute()``` function instead of the regular ```compute()``` method. Asynchronous calculation is a complex topic about which you can learn more e.g. [here](https://distributed.dask.org/en/latest/manage-computation.html).

In [ ]:
value = client.compute(maxval)

In [ ]:
value.result()

## Exercise

Open a new notebook and just try to create a cluster, a client, a dashboard, and run some code on it.